In [6]:
from xml.etree import ElementTree as ET

In [21]:
import pandas as pd

In [7]:
document_tree = ET.parse( 'data/mondial_database_less.xml' )

In [85]:
doc_tree = ET.parse( 'data/mondial_database.xml' )

#### 1. 10 countries with the lowest infant mortality rates

In [54]:
# empty dict to store 'country: mortality' values
countries = {}

# loop through the country elements and add mortality values for each country in the dict
for element in doc_tree.iterfind('country'):
    for inf_mort in element.iterfind('infant_mortality'):
        countries[element.find('name').text] = [float(element.find('infant_mortality').text)]

# create a DataFrame from the dict, sort showing lowest mortality rate first
countries_df = pd.DataFrame.from_dict(countries, orient = 'index')
countries_df.columns=['Infant Mortality']
countries_df.sort_values(by='Infant Mortality', ascending=True).head(10)


,Infant Mortality
Monaco,1.81
Japan,2.13
Norway,2.48
Bermuda,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


#### 2. 10 cities with the largest population

In [84]:
# create empty list to store 'country', 'city', 'year', 'population' values
city_pop = []
# loop through 'country' elements
for element in doc_tree.iterfind('country'):
    country = element.find('name').text
    # loop through city subelements
    for subelement in element.getiterator('city'):
        city = subelement.find('name')
        if city != None:
            yr = 0
            pop = 0
            # loop through 'population' subelements
            for popelement in subelement.getiterator('population'):          
                if popelement != None: 
                    # get the population value only from the most recent year
                    if int(popelement.attrib['year']) > yr:
                        yr = int(popelement.attrib['year'])
                        pop = int(popelement.text)
            # append the all the elemnts to the list            
            city_pop.append([country, city.text, int(popelement.attrib['year']), int(popelement.text)])

# create a DataFrame from the list, define columns, sort values to show largest population
city_pop_df = pd.DataFrame.from_records(city_pop)
city_pop_df.columns = ['Country', 'City', 'Year', 'Population']
city_pop_df.sort_values(by='Population', ascending=False).head(10)

,Country,City,Year,Population
1341,China,Shanghai,2010,22315474
771,Turkey,Istanbul,2012,13710512
1527,India,Mumbai,2011,12442373
479,Russia,Moskva,2013,11979529
1340,China,Beijing,2010,11716620
2810,Brazil,São Paulo,2010,11152344
1342,China,Tianjin,2010,11090314
1064,China,Guangzhou,2010,11071424
1582,India,Delhi,2011,11034555
1067,China,Shenzhen,2010,10358381


#### 3. 10 ethnic groups with the largest overall  populations (sum of best/latest estimates over all countries)

In [414]:
# create empty list to store values
eth_groups = []
# loop through the countries
for element in doc_tree.iterfind('country'):
    country = element.find('name').text
    year = 0
    #loop through population in each country from the most recent year
    for popelem in element.findall('population'):
        if int(popelem.attrib['year']) > year:
            population = int(popelem.text)
            year = int(popelem.attrib['year'])
    # loop through ethnic groups in each country
    # calculate ethnic groups' % ratio from overall country population
    for ethnic in element.findall('ethnicgroup'):
        percent = float(ethnic.attrib['percentage']) / 100
        eth_groups.append([country, ethnic.text, int(year), (float(population)*percent)])

# creata a DataFrame from the list, define columns
ethnic_gr_df = pd.DataFrame.from_records(eth_groups)
ethnic_gr_df.columns = ['Country', 'Ethnic Group', 'Year', 'Population']
# group by ethnic groups, aggregate pop values for grouped ethnic groups, sort to show largest pop values
ethnic_gr_df[['Ethnic Group', 'Population']].groupby(['Ethnic Group']).sum().sort_values(by='Population', ascending=False).head(10)

,Population
Ethnic Group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


#### 4. name and country of 
    a) longest river 
    b) largest lake 
    c) airport at highest elevation

In [363]:
# define a function that will return a DataFrame with desired subelement and its features
def search2(item, feature):
    
    # Create a dict to store 'country code: country name' pairs
    country_dict2={}
    for country_element in doc_tree.iterfind('country'):
        country_dict2[country_element.attrib['car_code']] = country_element.find('name').text

    # empty list to store subelement name, country code, country name, subelement feature
    item_country_list2 =[]
    # loop through the subelement to get the name and its feature 
    for item_el in doc_tree.iterfind(item):
        item2 = item_el.find('name').text
        item_feature = item_el.find(feature)
        if item_feature != None: # include only features that have value
            item_feature = item_feature.text
            # subelements may lay in several countries, so get each country code and add to the list
            for item_coun_code2 in item_el.attrib['country'].split():
                item_country2 = country_dict2[item_coun_code2]
                item_country_list2.append([item2, item_coun_code2, item_country2, float(item_feature)])
    
    # create a DataFrame from the list, define columns
    item_country_df2 = pd.DataFrame.from_records(item_country_list2)
    item_country_df2.columns=[item, 'CountryCode', 'CountryName', feature]

    df3 = item_country_df2.copy()
    # filter the rows to show only the desired item and its feature and associated countries
    measure = df3[item] == df3.sort_values(by=feature, ascending=False).iloc[0, 0]
    return df3[measure]

In [364]:
# longest river and countries
df3 = search2('river', 'length')

In [365]:
df3

,river,CountryCode,CountryName,length
298,Amazonas,CO,Colombia,6448.0
299,Amazonas,BR,Brazil,6448.0
300,Amazonas,PE,Peru,6448.0


In [369]:
# largest lake and countries
df4 = search2('lake', 'area')

In [370]:
df4

,lake,CountryCode,CountryName,area
68,Caspian Sea,R,Russia,386400.0
69,Caspian Sea,AZ,Azerbaijan,386400.0
70,Caspian Sea,KAZ,Kazakhstan,386400.0
71,Caspian Sea,IR,Iran,386400.0
72,Caspian Sea,TM,Turkmenistan,386400.0


In [434]:
# airport at highest elevation
def search3(item, feature):
    
    # Create a dict to store 'country code: country name' pairs
    country_dict2={}
    for country_element in doc_tree.iterfind('country'):
        country_dict2[country_element.attrib['car_code']] = country_element.find('name').text
    
    # list to store item name, country code, country name and feature
    item_country_list2 =[]
    # loop through the desired element, find the feature
    # associate the element with corresponding country code and country
    for item_el in doc_tree.iterfind(item):
        item_name = item_el.find('name').text
        item_country_code = item_el.attrib['country']
        item_country = country_dict2[item_country_code]
        item_feature_el = item_el.find(feature)
        if item_feature_el != None: # to include only feature that has value
            item_feature = item_feature_el.text
            if item_feature != None: # to exclude empty strings
                item_country_list2.append([item_name, item_country_code, item_country, float(item_feature)])
    
    # create DataFrame from the list
    item_country_df = pd.DataFrame.from_records(item_country_list2)
    item_country_df.columns=[item, 'CountryCode', 'CountryName', feature]
    
    df = item_country_df.copy()
    # filter the rows to show only the desired item and its feature and associated country(s)
    elev = df[item] == df.sort_values(by=feature, ascending=False).iloc[0, 0]
    return df[elev]

In [435]:
# airport and elevation
df6 = search3('airport', 'elevation')

In [436]:
df6

,airport,CountryCode,CountryName,elevation
80,El Alto Intl,BOL,Bolivia,4063.0
